In [117]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
URLPRI = "http://www.losmundialesdefutbol.com"
url = "http://www.losmundialesdefutbol.com/mundiales.php"

In [118]:
def returnWebObject(url):
    return BeautifulSoup(urlopen(url) , "lxml")

In [119]:
class Player:
    def __init__(self, name ,position, wasCaptain):
        self.name = name
        self.position = position
        self.wasCaptain = False
    def __str__(self):
        return self.name
    def __repr__(self):
        return self.name

In [137]:
class Team:
    def __init__(self , name , goal ):
        self.name = name
        self.goal = goal
        self.formation = None
        self.players = list()
    def getStringFromFormation(self):
        # Example 4-4-2
        pass

In [121]:
class Match:
    def __init__(self , firstTeam , secondTeam ,goals ,info):
        self.firstTeam = Team(firstTeam , goals[0])
        self.secondTeam = Team(secondTeam , goals[1])
        self.info = info

In [122]:

class WorldCup:
    def __init__(self , year , location , champion) :
        self.champion = champion
        self.location = location
        self.year = year
        self.countries = set()
        self.matches = list()
    def __str__(self):
        return "Copa del mundo del " + str(self.year) + " /Campeon " + self.champion + "/Lugar: " + self.location 
    def __repr__(self):
        return "Copa del mundo del " + str(self.year) + " /Campeon " + self.champion + "/Lugar: " + self.location 

In [123]:
pageListWorldSoccerCups = returnWebObject(URLPRI)
listWorldCups = list()
tableWordChampions = pageListWorldSoccerCups.find("div", {"class": "rd-100-40 rd-pad-0-l2"})
for x in tableWordChampions.find_all("tr"):
    if x.find("div", {"class" : "left" , "style":"width: 50% ; min-width: 100px"}):
        auxSplit = x.text.split()
        listWorldCups.append(WorldCup(int(auxSplit[0]) , auxSplit[1] ,auxSplit[2]))
listWorldCups = listWorldCups[::-1]
listWorldCups

[Copa del mundo del 2018 /Campeon -/Lugar: Rusia,
 Copa del mundo del 2014 /Campeon Alemania/Lugar: Brasil,
 Copa del mundo del 2010 /Campeon España/Lugar: Sudáfrica,
 Copa del mundo del 2006 /Campeon Italia/Lugar: Alemania,
 Copa del mundo del 2002 /Campeon //Lugar: Corea,
 Copa del mundo del 1998 /Campeon Francia/Lugar: Francia,
 Copa del mundo del 1994 /Campeon Unidos/Lugar: Estados,
 Copa del mundo del 1990 /Campeon Alemania/Lugar: Italia,
 Copa del mundo del 1986 /Campeon Argentina/Lugar: México,
 Copa del mundo del 1982 /Campeon Italia/Lugar: España,
 Copa del mundo del 1978 /Campeon Argentina/Lugar: Argentina,
 Copa del mundo del 1974 /Campeon Alemania/Lugar: Alemania,
 Copa del mundo del 1970 /Campeon Brasil/Lugar: México,
 Copa del mundo del 1966 /Campeon Inglaterra/Lugar: Inglaterra,
 Copa del mundo del 1962 /Campeon Brasil/Lugar: Chile,
 Copa del mundo del 1958 /Campeon Brasil/Lugar: Suecia,
 Copa del mundo del 1954 /Campeon Alemania/Lugar: Suiza,
 Copa del mundo del 1950 /C

In [124]:
pageListWorldSoccerCups = returnWebObject(url)

In [125]:
linksResultsWorlCupsMatches = []
for link in pageListWorldSoccerCups.findAll("a",href=True):
    if "resultados" in link["href"]:
        linksResultsWorlCupsMatches.append(link["href"])
linksResultsWorlCupsMatches

['mundiales/2018_resultados.php',
 'mundiales/2014_resultados.php',
 'mundiales/2010_resultados.php',
 'mundiales/2006_resultados.php',
 'mundiales/2002_resultados.php',
 'mundiales/1998_resultados.php',
 'mundiales/1994_resultados.php',
 'mundiales/1990_resultados.php',
 'mundiales/1986_resultados.php',
 'mundiales/1982_resultados.php',
 'mundiales/1978_resultados.php',
 'mundiales/1974_resultados.php',
 'mundiales/1970_resultados.php',
 'mundiales/1966_resultados.php',
 'mundiales/1962_resultados.php',
 'mundiales/1958_resultados.php',
 'mundiales/1954_resultados.php',
 'mundiales/1950_resultados.php',
 'mundiales/1938_resultados.php',
 'mundiales/1934_resultados.php',
 'mundiales/1930_resultados.php']

In [126]:
webPages = list()
for links in linksResultsWorlCupsMatches:
    webPages.append(returnWebObject(URLPRI+"/"+links))


In [127]:
def getGoals(result):
    goals = result.split("-")
    return int(goals[0]),int(goals[1])
def cleanWords(firsWord , secondWord = None):
    if secondWord == None :
        return firsWord .strip()
    else:
        return firsWord.strip() , secondWord.strip()

In [128]:
for i , worldCup in enumerate(webPages):    
    for row in worldCup.findAll("div",{"class": "margen-t3 clearfix"}): # Una linea de la pagina 
        infoMatch = row.find("div",{"class":"game margen-b3 clearfix"})
        firstTeam = infoMatch.find("div",{"class":"left margen-b1 clearfix"})
        secondTeam = infoMatch.find("div",{"class": "left a-left margen-b1 clearfix"})
        firstTeam , secondTeam = cleanWords(firstTeam.text , secondTeam.text)       
        listWorldCups[i].countries.add(secondTeam)
        listWorldCups[i].countries.add(firstTeam)
        if i != 0:
            for result in infoMatch.find("div",{"class" : "left a-center margen-b3 clearfix"}).findAll("a",href=True):
                continue
            listWorldCups[i].matches.append(Match(firstTeam , secondTeam , getGoals(result.text), result["href"]))
        else :
            listWorldCups[i].matches.append(Match(firstTeam , secondTeam , (0,0), ""))


In [129]:
listWorldCups = listWorldCups[1:] # No se esta considerando el mundial del 2018

In [130]:
def getLinkForMatch(info):
    #con esto obtenemos los links necesarios 
    if ".." in info:
        return URLPRI + info[2:]
    else :
        return URLPRI + info

In [131]:
def getPlayerInfo(row):
    name = ""
    wasCapitan = False
    pos = cleanWords(str(row.find("td").text))
    for x in row.find("div",{"style":"float: left ; "}).text.split():
        if x == "(C)":
            wasCapitan = True
        else:
            name = x + " " + name
    return Player(name, pos , wasCapitan)

In [142]:
for worldCups in listWorldCups[0:1]:
    for match in worldCups.matches[0:1]:
        contain = returnWebObject(getLinkForMatch(match.info))
        firstTable = contain.find("div",{"class":"rd-100-50 rd-pad-0-r2"})
        formation = dict()
        for i , row in enumerate(firstTable.findAll("tr",{"style":"vertical-align: top"})):
            if i > 10:
                break
            player = getPlayerInfo(row)
            try:
                formation[player.position] = formation[player.position] +1
            except:
                formation[player.position] = 1
            match.firstTeam.players.append(player)
        match.firstTeam.formation = formation
        secondTable = contain.find("div",{"class":"rd-100-50 rd-pad-0-l2"})
        formation = dict()
        for i , row in enumerate(secondTable.findAll("tr",{"style":"vertical-align: top"})):
            if i > 10:
                break
            player = getPlayerInfo(row)
            try:
                formation[player.position] = formation[player.position] +1
            except:
                formation[player.position] = 1
            match.secondTeam.players.append(player)
        match.secondTeam.formation = formation
            
            

In [141]:
listWorldCups[0].matches[0].secondTeam.players = list()
listWorldCups[0].matches[1].firstTeam.players = list()

In [143]:
listWorldCups[0].matches[0].secondTeam.players

[Pletikosa Stipe ,
 Vrsaljko Sime ,
 Corluka Vedran ,
 Lovren Dejan ,
 Srna Darijo ,
 Perisic Ivan ,
 Rakitic Ivan ,
 Modric Luka ,
 Kovacic Mateo ,
 Jelavic Nikica ,
 Olic Ivica ]